In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, LSTM
from keras.optimizers import RMSprop
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [2]:
# Load the data
df = pd.read_csv("dataA.csv")

# Add moving average
df['Moving_Avg'] = df['Close'].rolling(window=5).mean()

# Function to compute RSI
def compute_rsi(data, window):
    diff = data.diff()
    up_chg = 0 * diff
    down_chg = 0 * diff

    up_chg[diff > 0] = diff[ diff>0 ]
    down_chg[diff < 0] = diff[ diff < 0 ]

    up_chg_avg   = up_chg.rolling(window=window).mean()
    down_chg_avg = down_chg.rolling(window=window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

# Add RSI
df['RSI'] = compute_rsi(df['Close'], 5)
df = df.dropna()  # Drop NaN values
df.head()


,Symbol,Date,Open,High,Low,Close,percent_change,Volume,Moving_Avg,RSI
5,LBBL,07/08/2022,349.2,349.2,330.0,330.1,-4.60,36211,328.40,38.477801
6,LBBL,07/07/2022,360.0,368.0,343.0,346.0,-2.54,37061,328.80,51.661130
7,LBBL,07/06/2022,349.0,358.0,335.0,355.0,0.85,62733,332.20,63.449367
8,LBBL,07/05/2022,353.0,370.0,345.0,352.0,-2.22,60796,339.60,92.824074
9,LBBL,07/04/2022,370.0,382.0,350.0,360.0,-1.37,108253,348.62,94.129159


In [3]:
# Features and target variable
features = df[['Close', 'Moving_Avg', 'RSI']]
target = df['Close']

# Split data into 80-20 for training and testing
training_data_len = int(0.8 * len(features))
train_features = features[:training_data_len]
train_target = target[:training_data_len]
test_features = features[training_data_len:]
test_target = target[training_data_len:]

# Scale the features
scaler_features = StandardScaler()
train_features_scaled = scaler_features.fit_transform(train_features)
test_features_scaled = scaler_features.transform(test_features)

# Scale the target
scaler_target = StandardScaler()
train_target_scaled = scaler_target.fit_transform(train_target.values.reshape(-1, 1))
test_target_scaled = scaler_target.transform(test_target.values.reshape(-1, 1))

# Create data structure with 5 timestamps for training and testing data
window_size = 5
x_train, y_train, x_test, y_test = [], [], [], []

for i in range(window_size, len(train_features_scaled)):
    x_train.append(train_features_scaled[i-window_size:i])
    y_train.append(train_target_scaled[i])
for i in range(window_size, len(test_features_scaled)):
    x_test.append(test_features_scaled[i-window_size:i])
    y_test.append(test_target_scaled[i])

x_train, y_train, x_test, y_test = np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)


In [4]:
# Define the GRU Model
gru_model = Sequential()
gru_model.add(GRU(units=50, return_sequences=True, input_shape=(x_train.shape[1], 3)))
gru_model.add(Dropout(0.2))
gru_model.add(GRU(units=75, return_sequences=False))
gru_model.add(Dropout(0.2))
gru_model.add(Dense(units=20))
gru_model.add(Dense(units=1))
gru_model.compile(optimizer=RMSprop(learning_rate=0.001), loss='mean_squared_error')

# Train the model
gru_history = gru_model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/50
108/108 [==============================] - 14s 36ms/step - loss: 0.0338 - val_loss: 0.0113
Epoch 2/50
108/108 [==============================] - 2s 14ms/step - loss: 0.0203 - val_loss: 0.0327
Epoch 3/50
108/108 [==============================] - 1s 13ms/step - loss: 0.0196 - val_loss: 0.0083
Epoch 4/50
108/108 [==============================] - 2s 14ms/step - loss: 0.0181 - val_loss: 0.0149
Epoch 5/50
108/108 [==============================] - 1s 13ms/step - loss: 0.0169 - val_loss: 0.0080
Epoch 6/50
108/108 [==============================] - 1s 13ms/step - loss: 0.0164 - val_loss: 0.0094
Epoch 7/50
108/108 [==============================] - 2s 14ms/step - loss: 0.0179 - val_loss: 0.0129
Epoch 8/50
108/108 [==============================] - 1s 14ms/step - loss: 0.0159 - val_loss: 0.0059
Epoch 9/50
108/108 [==============================] - 1s 14ms/step - loss: 0.0153 - val_loss: 0.0063
Epoch 10/50
108/108 [==============================] - 1s 13ms/step - loss: 0.0140 - val_l

In [5]:
# Define the Stacked LSTM Model
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 3)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=75, return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(units=20))
lstm_model.add(Dense(units=1))
lstm_model.compile(optimizer=RMSprop(learning_rate=0.001), loss='mean_squared_error')

# Train the model
lstm_history = lstm_model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/50
108/108 [==============================] - 16s 42ms/step - loss: 0.0639 - val_loss: 0.0345
Epoch 2/50
108/108 [==============================] - 2s 19ms/step - loss: 0.0316 - val_loss: 0.0170
Epoch 3/50
108/108 [==============================] - 2s 16ms/step - loss: 0.0281 - val_loss: 0.0162
Epoch 4/50
108/108 [==============================] - 2s 17ms/step - loss: 0.0287 - val_loss: 0.0166
Epoch 5/50
108/108 [==============================] - 2s 16ms/step - loss: 0.0266 - val_loss: 0.0244
Epoch 6/50
108/108 [==============================] - 2s 17ms/step - loss: 0.0249 - val_loss: 0.0212
Epoch 7/50
108/108 [==============================] - 2s 17ms/step - loss: 0.0244 - val_loss: 0.0203
Epoch 8/50
108/108 [==============================] - 2s 19ms/step - loss: 0.0233 - val_loss: 0.0474
Epoch 9/50
108/108 [==============================] - 2s 19ms/step - loss: 0.0234 - val_loss: 0.0348
Epoch 10/50
108/108 [==============================] - 2s 17ms/step - loss: 0.0224 - val_l